# Script to regrid ILAMB datasets to 2degree CLM-PPE grid
- Updated Feb 24, 2023
- Linnia Hawkins, lh3194@columbia.edu

Requirements: xesmf https://xesmf.readthedocs.io/en/latest/index.html
Conda env: xesmf_env

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import glob
import os
import xarray as xr
import xesmf as xe

In [2]:
def regrid_ilamb(ds, variable, dataset, method="bilinear"):
    # ds needs to have dimensions labeled 'lat' and 'lon'

    # Load target grid
    ds_sp = xr.open_dataset('/glade/campaign/cgd/tss/people/oleson/PPE/' +
                            'clm51_lmbirchwkattgen01ctsm51d006_2deg_GSWP3V1' +
                            '_nocrop_PPE2_hist/lnd/hist/' +
                            'clm51_lmbirchwkattgen01ctsm51d006_2deg_GSWP3V1_' +
                            'nocrop_PPE2_hist.clm2.h0.2014-12.nc')
    target_grid = ds_sp.TLAI.mean(dim='time')
    target_grid['mask'] = ds_sp.landmask

    regridder = xe.Regridder(ds, target_grid, method)
    in_var = ds[variable]
    in_var["mask"] = xr.where(~np.isnan(in_var), 1, 0)
    ds_out = regridder(in_var)
    ds_out = ds_out*ds_sp.landmask

    return ds_out

In [3]:
def regrid_dataset(dir, model, dataset, variable):

    f = os.path.join("/glade/work/afoster/FATES_calibration/ILAMB_data/regridded",
                     f"{model}_{variable}_regridded.nc")
    var = variable.lower()
    ds = xr.open_dataset(os.path.join(dir, dataset))
    dr_out = regrid_ilamb(ds, var, dataset)
    dsout = xr.Dataset()
    dsout[var] = dr_out
    dsout[var].attrs = ds[var].attrs
    dsout.to_netcdf(f)

In [4]:
# ILAMB directory
ILAMB_dir = '/glade/work/afoster/FATES_calibration/ILAMB_data'

In [7]:
model = 'CERES'
var = 'albedo'
file = f"/glade/work/afoster/FATES_calibration/ILAMB_data/{model}_{var}.nc"
regrid_dataset(ILAMB_dir, model, file, var)

In [6]:
for file in os.listdir(ILAMB_dir):
    sep = file.split('_')
    if len(sep) > 1:
        if '.nc' in sep[1]:
            regrid_dataset(ILAMB_dir, sep[0], file, sep[1].replace('.nc', ''))